In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
import os 
# kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"
# spark_k = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"
kafka_spark = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {}  pyspark-shell".format(kafka_spark))

In [3]:
spark = SparkSession.builder.master("local").appName("process-tweet-sentiment").getOrCreate()

In [4]:
spark

In [31]:
KAFKA_BROKER = "localhost:9092"
KAFKA_TOPIC_CONSUMER = "twitter-sentiment"
df = spark.readStream.\
        format("kafka").\
        option("kafka.bootstrap.servers",KAFKA_BROKER).\
        option("subscribe",KAFKA_TOPIC_CONSUMER).\
        option("startingOffsets","earliest").\
        load()

In [32]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [33]:
def evaluate(avg):
    try:
        if avg < 0:
            return 'Negative'
        elif avg >0:
            return 'Positive'
        else:
            return 'Neutral'
    except:
        return 'Neutral'


In [34]:
schema = StructType([StructField("id",StringType()),StructField("time",StringType()),StructField("text",StringType()),StructField("sentiment_score",DoubleType())])

In [35]:
#filter all hashtag in each tweet by timestamp
processed_tweet_value = df.selectExpr("CAST(value as string) AS value").\
                    select(from_json("value",schema).alias("x")).\
                    selectExpr("CAST(from_unixtime(x.time,'HH:mm:ss') as TIMESTAMP) AS time",
                              "x.text as text",
                              "x.sentiment_score as sentiment_score")
                    

In [39]:
eval_udf = udf(evaluate,StringType())
processed_tweet_value = processed_tweet_value.withColumn("status",eval_udf("sentiment_score"))
#de ghi vao kafka tiep thi phai co mot cot la value voi gia tri la json
target_df = processed_tweet_value.selectExpr("to_json(struct(*)) as value")

In [30]:
processed_tweet_value.writeStream.\
                outputMode("update").\
                format("console").\
                option("truncate","false").\
                start()

In [ ]:
nifi_query = target_df\
        .writeStream \
        .queryName("Notification Writer") \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BROKER) \
        .option("topic", "twitter-sentiment-2") \
        .outputMode("update") \
        .option("checkpointLocation", "chk-point-dir") \
        .start()

nifi_query.awaitTermination()
